In [ ]:
from google.colab import drive
drive.mount('/content/drive' )

## Setup

In [1]:
import os
import pathlib


# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3205, done.
remote: Counting objects: 100% (3205/3205), done.
remote: Compressing objects: 100% (2707/2707), done.
remote: Total 3205 (delta 855), reused 1372 (delta 454), pack-reused 0
Receiving objects: 100% (3205/3205), 33.40 MiB | 26.15 MiB/s, done.
Resolving deltas: 100% (855/855), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Graph to Saved Model

In [19]:
#image to np array
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

## Create Representative Dataset

-- TODO: load images in array so they can be recognized by tflite quantizer : see tf signatures

appraoch 1: singature (reccomended)

In [42]:
def representative_dataset():
  for data in dataset:
    yield {
      "image": data.image,
      "bias": data.bias,
    }

approach 2: input tensor list

In [ ]:
def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((images)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]

appraoch 3: numpy array

In [24]:
dataset = load_image_into_numpy_array('/content/drive/MyDrive/School/CSU/Senior_Year/ECE_528/ECE528_Project/dataset/train/images/face18.jpg')

In [51]:
def representative_dataset():
  data = dataset
  num_calibration_steps = 1
  for _ in range(num_calibration_steps):
    image, = data
    yield [data]

## Full Integer Quantization


In [52]:
saved_model_dir = '/content/drive/MyDrive/School/CSU/Senior_Year/ECE_528/ECE528_Project/effDet_output/saved_model'

In [53]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

ValueError: ignored